<a href="https://colab.research.google.com/github/tony23stark/tweetly-ai/blob/main/Bebo_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [ ]:
!pip install -qU langchain-google-genai langchain langgraph google-generativeai openai


# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage



# Secure Gemini API Key input

In [ ]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Enter Gemini API Key: ')

Enter Gemini API Key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature =0.2)

# STEP 3: Node to ask user for symptom

In [ ]:
def get_symptom(state: dict) ->dict:
  symptom =input("Welome to XYZ hospital, Please enter your symptom")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [ ]:
def classify_symptom(state: dict) -> dict:
  prompt =(
      "you are a helpful medical assistant, classify the symptoms below into one of the categories  \n"
      "General\n -Emergence \n -mental health \n"
      f"symptom : {state['symptom']} \n"
      "Respond only with one word : General, Emergency Or Mental Health"
      "#Example : input : I have fever,Output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as :(category)")  #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [ ]:
def symptom_router(state: dict) -> dict:
  cat = state["category"].lower() #General,general,Mental,mental
  if "general" in cat:
    return"general"
  elif "emergency" in cat:
      return "emergency"
  elif "mental" in cat:
      return "mental_health"
  else:
        return"general"

# STEP 6: Category-specific response nodes


In [ ]:
def general_node(state: dict) -> dict:
    symptom = state.get("symptom", "No symptom provided")
    state["answer"] = f"'{symptom}' : seems general - directing you to general ward for consulting a doctor"
    return state

def emergency_node(state: dict) -> dict:
    symptom = state.get("symptom", "No symptom provided")
    state["answer"] = f"'{symptom}' : It is a Medical Emergency - seeking immediate help"
    return state

def mental_health_node(state: dict) -> dict:
    symptom = state.get("symptom", "No symptom provided")
    state["answer"] = f"'{symptom}' : seems like a mental health issue - talk to our counsellor"
    return state


# STEP 7: Build LangGraph


In [ ]:
builder = StateGraph(dict)

# Define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

# Add edge from get_symptom to classify
builder.add_edge("get_symptom", "classify")  # corrected from builder.add.edge

# Add conditional edges based on router function
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental_health": "mental_health"
})

# End each path
builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)


# STEP 8: Compile and invoke the graph


In [ ]:
graph = builder.compile()
final_state = graph.invoke({})
print("\nFinal Output:\n")
print(final_state.get("answer", "No answer generated"))


Welome to XYZ hospital, Please enter your symptomwanna cry
LLM classifies the symptom as :(category)

Final Output:

'wanna cry' : seems like a mental health issue - talk to our counsellor
